In [ ]:
import os
import torch
import pandas as pd
import scanpy as sc
import squidpy as sq
import numpy as np


In [2]:
import SpatialGlue_3M

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# the location of R, which is required for the 'mclust' algorithm. Please replace the path below with local R installation path
os.environ['R_HOME'] = 'C:\Program Files\R\R-4.4.0'

In [ ]:
# read data
#file_fold = '/home/yahui/anaconda3/work/SpatialGlue_revision/data/Dataset11_Lymph_Node_A1/' #please replace 'file_fold' with the download path

adata_omics1 = sc.read_h5ad('./data/10x Visium human tonsil/rna.h5ad')
adata_omics2 = sc.read_h5ad('./data/10x Visium human tonsil/protein.h5ad')
adata_omics3 = sc.read_h5ad('./data/10x Visium human tonsil/protein.h5ad')
adata_omics3.obsm['image_feature']=np.load('./data/10x Visium human tonsil/image_emb.npy')

loc = pd.read_csv('./data/10x Visium human tonsil/locs.csv')
spatial_coords = loc.iloc[:, [2, 3]].values  
adata_omics1.obsm['spatial'] = spatial_coords
adata_omics2.obsm['spatial'] = spatial_coords
adata_omics3.obsm['spatial'] = spatial_coords

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()
adata_omics3.var_names_make_unique()

d:\anaconda\envs\SpatialGlue\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
d:\anaconda\envs\SpatialGlue\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
d:\anaconda\envs\SpatialGlue\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [9]:
# Fix random seed
from SpatialGlue_3M.preprocess import fix_seed
random_seed = 2022
fix_seed(random_seed)

In [ ]:
#from SpatialGlue_3M.preprocess import clr_normalize_each_cell, pca
from SpatialGlue.preprocess import clr_normalize_each_cell, pca

from sklearn.decomposition import PCA
n_protein = adata_omics2.n_vars

sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)

adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
adata_omics1.obsm['feat'] = pca(adata_omics1_high, n_comps=n_protein)

# Protein
adata_omics2 = clr_normalize_each_cell(adata_omics2)
adata_omics2.obsm['feat'] = pca(adata_omics2, n_comps=n_protein)

# Image
adata_omics3.obsm['feat']=PCA(n_components=n_protein, random_state=2022).fit_transform(adata_omics3.obsm['image_feature'])

In [ ]:
from SpatialGlue_3M.preprocess import construct_neighbor_graph
data = construct_neighbor_graph(adata_omics1, adata_omics2, adata_omics3)

In [ ]:
# define model
from SpatialGlue_3M.SpatialGlue_pyG import Train_SpatialGlue
model = Train_SpatialGlue(data, device=device, epochs=200)

# train model
output = model.train()

In [ ]:
adata = adata_omics1.copy()
# extract output labels
adata.obsm['SpatialGlue'] = output['SpatialGlue']

In [ ]:
# We set 'mclust' as clustering tool by default. Users can also select 'leiden' and 'louvain'
from SpatialGlue.utils import clustering
tool = 'mclust' # mclust, leiden, and louvain
clustering(adata, key='SpatialGlue', add_key='SpatialGlue', n_clusters=10, method=tool, use_pca=True)

In [ ]:
sc.pl.embedding(adata, basis='spatial', color='SpatialGlue', title='SpatialGlue', s=90, show=False)